# Model training for 3-segment reconstructions

## Import

In [183]:
import warnings
import os
import numpy as np
import pandas as pd
import copy
from statistics import mean, stdev
from sklearn.preprocessing import QuantileTransformer, RobustScaler
from sklearn.metrics import make_scorer, matthews_corrcoef, f1_score, accuracy_score, average_precision_score, roc_auc_score, brier_score_loss
from sklearn.model_selection import cross_validate, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from scipy.stats import chi2_contingency, chi2, ttest_ind, barnard_exact, fisher_exact
import shap
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import seaborn as sns
import matplotlib.pyplot as plt

In [184]:
df_dropped_first = pd.read_parquet('/Users/philipp.lampert/repositories/mymandible/data/dropped_first_imputed.parquet')
df_all_levels = pd.read_parquet('/Users/philipp.lampert/repositories/mymandible/data/all_levels_imputed.parquet')

In [185]:
def get_3_segment_df(df):
    df3 = df[
        (df['flap_segment_count'] == 3) 
        & (df['flap_donor_site___scapula'] == False) 
        & (df['plate_type___cad_mini'] == False)
        & (df['indication___secondary_reconstruction'] == False)
        & (df['indication___osteoradionecrosis'] == False)
        & (df['days_to_follow_up'] > 182)
        & (df['urkens_classification___r'] == False)
    ].copy()
    
    df3['follow_up_completed'] = df3['days_to_follow_up'] > 182
    
    flap_loss_mix = df3[(df3['plate_type___cad_mix'] == True) & (df3['days_to_flap_loss'] < 183)]['flap_loss'].sum()
    flap_loss_reco = df3[(df3['plate_type___cad_mix'] == False) & (df3['days_to_flap_loss'] < 183)]['flap_loss'].sum()

    print("Number of flap loss within 6 months for mixed plates:", flap_loss_mix)
    print("Number of flap loss within 6 months for reco plates:", flap_loss_reco)
    
    df3 = df3[~((df3['flap_loss'] == True) & (df3['days_to_flap_loss'] < 183))]

    # drop unused variables
    df3 = df3.drop(['flap_donor_site___scapula', 
                    'flap_segment_count', 
                    'plate_type___cad_mini', 
                    'urkens_classification___s',
                    'indication___osteoradionecrosis',
                    'indication___secondary_reconstruction',
                    'prior_flap___bony'
                   ], axis=1)
    return df3

In [186]:
df_df = get_3_segment_df(df_dropped_first)
df_all = get_3_segment_df(df_all_levels)

Number of flap loss within 6 months for mixed plates: 0
Number of flap loss within 6 months for reco plates: 1
Number of flap loss within 6 months for mixed plates: 0
Number of flap loss within 6 months for reco plates: 1


In [187]:
df_df[df_df['nonunion'].isna()]

,id,sex_female,comorbidity___smoking,comorbidity___alcohol,comorbidity___copd,comorbidity___hypertension,comorbidity___diabetes,comorbidity___atherosclerosis,comorbidity___hyperlipidemia,comorbidity___hypothyroidism,...,days_to_imaging_12_24,nonunion_12_24,nonunion_12_24_location___mandible_flap,nonunion_12_24_location___flap_flap,soft_tissue_complication,plate_failure,orn,nonunion,any_complication,follow_up_completed
67,68,False,True,False,False,False,False,False,False,False,...,<NA>,<NA>,False,False,True,False,True,<NA>,True,True
87,88,False,True,True,False,False,False,False,False,False,...,<NA>,<NA>,False,False,True,False,False,<NA>,True,True
102,103,False,True,True,False,False,False,False,False,False,...,<NA>,<NA>,False,False,True,False,False,<NA>,True,True
134,135,False,True,True,True,False,False,False,False,False,...,<NA>,<NA>,False,False,False,False,False,<NA>,False,True
203,204,False,False,False,False,False,False,False,False,False,...,<NA>,<NA>,False,False,True,False,False,<NA>,True,True


## Preprocessing

In [188]:
from modules.functions import preprocessing as prp
from modules.functions import threshold_optimized_metrics as tom

In [189]:
acc_scorer = make_scorer(tom.optimized_accuracy, needs_proba=True)
f1_scorer = make_scorer(tom.optimized_f1, needs_proba=True)
mcc_scorer = make_scorer(tom.optimized_mcc, needs_proba=True)
pr_auc_scorer = make_scorer(average_precision_score, needs_proba=True)

## Model setup

In [190]:
def logreg_regularized(outcome, scaler, df, method, alpha):
    
    x, y = prp.get_x_y(df=df, outcome=outcome, min_follow_up_days=91, scaler=scaler, drop_cols=drop_cols, inverse_pos=False)
    boolean_columns = x.select_dtypes(include=bool).columns
    x[boolean_columns] = x[boolean_columns].astype('int')
    numeric_columns = x.select_dtypes(include='number').columns
    x[numeric_columns] = x[numeric_columns].astype('float64')
    y = y.astype('int')    
    x_columns = x.columns
    all_columns = "+".join(x_columns)
    formula = outcome +  '~' + all_columns
    
    data = pd.concat([x, y], axis=1)
    final_model = smf.logit(formula, data).fit_regularized(method=method, alpha=alpha)
    print(final_model.summary())

##### Export Parquet data for Exact Logistic Regression in R

In [191]:
drop_cols = [
    'id',
    #'sex_female', 
    #'comorbidity___smoking', 
    #'comorbidity___alcohol',
    'comorbidity___copd', 
    'comorbidity___hypertension',
    'comorbidity___diabetes', 
    #'comorbidity___atherosclerosis',
    'comorbidity___vascular_disease',
    'comorbidity___hyperlipidemia', 
    'comorbidity___hypothyroidism',
    'comorbidity___chronic_kidney_disease',
    'comorbidity___autoimmune_disease', 
    #'age_surgery_years',
    'radiotherapy___pre_surgery', 
    #'radiotherapy___post_surgery',
    'chemotherapy___pre_surgery', 
    #'chemotherapy___post_surgery',
    'urkens_classification___c', 
    'urkens_classification___r',
    'surgery_duration_min', 
    #'bmi', 
    #'skin_transplanted',
    'prior_flap___non_bony', 
    #'plate_type___cad_mix'
]

In [192]:
def export_parquet(outcome, df):
    x, y = prp.get_x_y(df=df, outcome=outcome, min_follow_up_days=91, scaler='None', drop_cols=drop_cols, inverse_pos=False)
    df = pd.concat([x, y], axis=1)
    df.to_parquet(f"{outcome}_3_seg.parquet")
    destination_path = "/Users/philipp.lampert/repositories/mymandible/data/outcome_df/"
    os.rename(f"{outcome}_3_seg.parquet", os.path.join(destination_path, f"{outcome}_3_seg.parquet"))

In [193]:
export_parquet('wound_infection', df_df)

## Patient characteristics

### Numeric variable

In [194]:
def num_variable(variable):
    cad_mix_values = df_df.loc[df_df['plate_type___cad_mix'], variable]
    cad_long_values = df_df.loc[~df_df['plate_type___cad_mix'], variable]
    
    cad_mix_mean = round(cad_mix_values.mean(), 1)
    cad_mix_std = round(cad_mix_values.std(), 1)
    cad_long_mean = round(cad_long_values.mean(), 1)
    cad_long_std = round(cad_long_values.std(), 1)
    
    overall_mean = round(df_df[variable].mean(), 1)
    overall_std = round(df_df[variable].std(), 1)
    
    data = {
        'Mix': [cad_mix_mean, cad_mix_std],
        'Reco': [cad_long_mean, cad_long_std],
        'Overall': [overall_mean, overall_std]
    }
    
    df = pd.DataFrame(data, index=['mean', 'std'])
    print(df)

In [195]:
def t_test(variable, df):

    df_mix = df[df['plate_type___cad_mix'] == True]
    df_reco = df[df['plate_type___cad_mix'] == False]
    array_mix = df_mix[variable].values.astype('int')
    array_reco = df_reco[variable].values.astype('int')
    t_test = ttest_ind(a=array_mix, b=array_reco, nan_policy='raise')
    print(num_variable(variable))
    print(f"p-value: {t_test.pvalue}")

### Nominal variables

In [196]:
def cat_variable(variable):
    # Absolute frequencies
    cad_mix_counts = df_df.loc[df_df['plate_type___cad_mix'], variable].value_counts()
    cad_long_counts = df_df.loc[~df_df['plate_type___cad_mix'], variable].value_counts()
    overall_counts = df_df[variable].value_counts()
    
    # Relative probabilities
    cad_mix_probs = round((cad_mix_counts / cad_mix_counts.sum())*100, 1)
    cad_long_probs = round((cad_long_counts / cad_long_counts.sum())*100, 1)
    overall_probs = round((overall_counts / overall_counts.sum())*100, 1)
    
    # Create DataFrames for absolute frequencies and relative probabilities
    absolute_freq_df = pd.DataFrame({
        'Mix': cad_mix_counts,
        'Reco': cad_long_counts,
        'Overall': overall_counts
    }).fillna(0)  # Fill NaN values with 0
    
    relative_prob_df = pd.DataFrame({
        'Mix': cad_mix_probs,
        'Reco': cad_long_probs,
        'Overall': overall_probs
    }).fillna(0)  # Fill NaN values with 0
    
    print("Absolute frequencies:")
    print(absolute_freq_df)
    
    print("\nRelative probabilities:")
    print(relative_prob_df)

In [197]:
def chi2_test(variable, df):
    contingency = pd.crosstab(df[variable], df['plate_type___cad_mix'])
    chi2_object = chi2_contingency(observed=contingency, correction=False)
    N = contingency.sum().sum()
    n_minus_1 = chi2_object.statistic * (N - 1) / N
    p_value = chi2.sf(n_minus_1, 1)
    print(cat_variable(variable))
    print(f"p-value: {p_value}")

In [198]:
def barnard_test(variable, df):
    contingency = pd.crosstab(df[variable], df['plate_type___cad_mix'])
    obj = barnard_exact(contingency)
    print(cat_variable(variable))
    print(f"p-value: {obj.pvalue}")

In [199]:
def fisher_test(variable, df):
    contingency = pd.crosstab(df[variable], df['plate_type___cad_mix'])
    obj = fisher_exact(contingency)
    print(cat_variable(variable))
    print(f"p-value: {obj.pvalue}")

### Results

In [200]:
t_test('age_surgery_years', df_df)

       Mix  Reco  Overall
mean  60.6  62.0     61.4
std    7.6   7.8      7.6
None
p-value: 0.5955299274573307


In [201]:
chi2_test('sex_female', df_df)

Absolute frequencies:
            Mix  Reco  Overall
sex_female                    
False        11    17       28
True          3     7       10

Relative probabilities:
             Mix  Reco  Overall
sex_female                     
False       78.6  70.8     73.7
True        21.4  29.2     26.3
None
p-value: 0.6061233213607875


In [202]:
t_test('bmi', df_df)

            Mix  Reco  Overall
mean  23.700001  23.6     23.6
std    2.300000   4.7      3.9
None
p-value: 0.9301029646165078


In [203]:
chi2_test('comorbidity___smoking', df_df)

Absolute frequencies:
                       Mix  Reco  Overall
comorbidity___smoking                    
False                    8     9       17
True                     6    15       21

Relative probabilities:
                        Mix  Reco  Overall
comorbidity___smoking                     
False                  57.1  37.5     44.7
True                   42.9  62.5     55.3
None
p-value: 0.24639351297433254


In [204]:
chi2_test('comorbidity___alcohol', df_df)

Absolute frequencies:
                       Mix  Reco  Overall
comorbidity___alcohol                    
False                   10    15       25
True                     4     9       13

Relative probabilities:
                        Mix  Reco  Overall
comorbidity___alcohol                     
False                  71.4  62.5     65.8
True                   28.6  37.5     34.2
None
p-value: 0.5807987953691733


In [205]:
chi2_test('comorbidity___atherosclerosis', df_df)

Absolute frequencies:
                               Mix  Reco  Overall
comorbidity___atherosclerosis                    
False                           12    18       30
True                             2     6        8

Relative probabilities:
                                Mix  Reco  Overall
comorbidity___atherosclerosis                     
False                          85.7  75.0     78.9
True                           14.3  25.0     21.1
None
p-value: 0.44062934980147084


In [206]:
chi2_test('radiotherapy___post_surgery', df_df)

Absolute frequencies:
                             Mix  Reco  Overall
radiotherapy___post_surgery                    
True                           8    15       23
False                          6     9       15

Relative probabilities:
                              Mix  Reco  Overall
radiotherapy___post_surgery                     
True                         57.1  62.5     60.5
False                        42.9  37.5     39.5
None
p-value: 0.7477680079698745


In [207]:
chi2_test('chemotherapy___post_surgery', df_df)

Absolute frequencies:
                             Mix  Reco  Overall
chemotherapy___post_surgery                    
False                         10    12       22
True                           4    12       16

Relative probabilities:
                              Mix  Reco  Overall
chemotherapy___post_surgery                     
False                        71.4  50.0     57.9
True                         28.6  50.0     42.1
None
p-value: 0.20284749555189324


In [208]:
chi2_test('skin_transplanted', df_df)

Absolute frequencies:
                   Mix  Reco  Overall
skin_transplanted                    
False                8     8       16
True                 6    16       22

Relative probabilities:
                    Mix  Reco  Overall
skin_transplanted                     
False              57.1  33.3     42.1
True               42.9  66.7     57.9
None
p-value: 0.15707476398631248


In [209]:
chi2_test('follow_up_completed', df_df)

Absolute frequencies:
                     Mix  Reco  Overall
follow_up_completed                    
True                  14    24       38

Relative probabilities:
                       Mix   Reco  Overall
follow_up_completed                       
True                 100.0  100.0    100.0
None
p-value: 1.0


## Logistic Regression

#### VIF

In [210]:
vif_df, y = prp.get_x_y(df=df_df, outcome='any_complication', min_follow_up_days=91, scaler='None', drop_cols=drop_cols, inverse_pos=False)
boolean_columns = vif_df.select_dtypes(include=bool).columns
vif_df[boolean_columns] = vif_df[boolean_columns].astype('int')
numeric_columns = vif_df.select_dtypes(include='number').columns
vif_df[numeric_columns] =vif_df[numeric_columns].astype('float64')
X = add_constant(vif_df)
pd.Series([variance_inflation_factor(X.values, i) 
               for i in range(X.shape[1])], 
              index=X.columns)

const                            130.364448
sex_female                         1.190676
comorbidity___smoking              1.369553
comorbidity___alcohol              1.477339
comorbidity___atherosclerosis      1.510058
age_surgery_years                  1.275705
radiotherapy___post_surgery        1.666030
chemotherapy___post_surgery        1.939070
bmi                                1.267858
skin_transplanted                  1.422372
plate_type___cad_mix               1.252540
dtype: float64

### Any complication

In [211]:
df_df['any_complication'].value_counts()

any_complication
True     30
False     8
Name: count, dtype: Int64

In [212]:
logreg_regularized('any_complication', 'None', df_df, 'l1', alpha=0)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3808234793434244
            Iterations: 67
            Function evaluations: 71
            Gradient evaluations: 67
                           Logit Regression Results                           
Dep. Variable:       any_complication   No. Observations:                   38
Model:                          Logit   Df Residuals:                       27
Method:                           MLE   Df Model:                           10
Date:                Tue, 16 Apr 2024   Pseudo R-squ.:                  0.2600
Time:                        18:29:13   Log-Likelihood:                -14.471
converged:                       True   LL-Null:                       -19.557
Covariance Type:            nonrobust   LLR p-value:                    0.4256
                                    coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------

overflow encountered in exp
divide by zero encountered in log


### Soft tissue complication

In [213]:
df_df['soft_tissue_complication'].value_counts()

soft_tissue_complication
True     23
False    15
Name: count, dtype: Int64

In [214]:
logreg_regularized('soft_tissue_complication', QuantileTransformer(output_distribution='normal'), df_df, 'l1', alpha=0)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.4664530126714283
            Iterations: 58
            Function evaluations: 58
            Gradient evaluations: 58
                              Logit Regression Results                              
Dep. Variable:     soft_tissue_complication   No. Observations:                   38
Model:                                Logit   Df Residuals:                       27
Method:                                 MLE   Df Model:                           10
Date:                      Tue, 16 Apr 2024   Pseudo R-squ.:                  0.3047
Time:                              18:29:13   Log-Likelihood:                -17.725
converged:                             True   LL-Null:                       -25.491
Covariance Type:                  nonrobust   LLR p-value:                    0.1138
                                    coef    std err          z      P>|z|      [0.025      0.975]
------------

n_quantiles (1000) is greater than the total number of samples (38). n_quantiles is set to n_samples.


### Nonunion

In [215]:
df_df['nonunion'].value_counts()

nonunion
True     19
False    14
Name: count, dtype: Int64

In [216]:
logreg_regularized('nonunion', QuantileTransformer(output_distribution='normal'), df_df, 'l1', alpha=0)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6290043884868978
            Iterations: 52
            Function evaluations: 52
            Gradient evaluations: 52
                           Logit Regression Results                           
Dep. Variable:               nonunion   No. Observations:                   33
Model:                          Logit   Df Residuals:                       22
Method:                           MLE   Df Model:                           10
Date:                Tue, 16 Apr 2024   Pseudo R-squ.:                 0.07720
Time:                        18:29:13   Log-Likelihood:                -20.757
converged:                       True   LL-Null:                       -22.494
Covariance Type:            nonrobust   LLR p-value:                    0.9680
                                    coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------

n_quantiles (1000) is greater than the total number of samples (33). n_quantiles is set to n_samples.


### Wound infection

In [217]:
df_df['wound_infection'].value_counts()

wound_infection
False    25
True     13
Name: count, dtype: Int64

In [218]:
logreg_regularized('wound_infection', QuantileTransformer(output_distribution='normal'), df_df, 'l1', alpha=0)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.4312572412469582
            Iterations: 58
            Function evaluations: 58
            Gradient evaluations: 58
                           Logit Regression Results                           
Dep. Variable:        wound_infection   No. Observations:                   38
Model:                          Logit   Df Residuals:                       27
Method:                           MLE   Df Model:                           10
Date:                Tue, 16 Apr 2024   Pseudo R-squ.:                  0.3287
Time:                        18:29:13   Log-Likelihood:                -16.388
converged:                       True   LL-Null:                       -24.412
Covariance Type:            nonrobust   LLR p-value:                   0.09825
                                    coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------

n_quantiles (1000) is greater than the total number of samples (38). n_quantiles is set to n_samples.


In [219]:
np.exp(1.3061)

3.691747783578271

### Plate exposure

In [220]:
df_df['complication_plate___exposure'].value_counts()

complication_plate___exposure
False    26
True     12
Name: count, dtype: Int64

In [221]:
logreg_regularized('complication_plate___exposure', QuantileTransformer(output_distribution='normal'), df_df, 'l1', alpha=0)

n_quantiles (1000) is greater than the total number of samples (38). n_quantiles is set to n_samples.


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.4779827720376916
            Iterations: 57
            Function evaluations: 58
            Gradient evaluations: 57
                                 Logit Regression Results                                
Dep. Variable:     complication_plate___exposure   No. Observations:                   38
Model:                                     Logit   Df Residuals:                       27
Method:                                      MLE   Df Model:                           10
Date:                           Tue, 16 Apr 2024   Pseudo R-squ.:                  0.2336
Time:                                   18:29:13   Log-Likelihood:                -18.163
converged:                                  True   LL-Null:                       -23.699
Covariance Type:                       nonrobust   LLR p-value:                    0.3520
                                    coef    std err          z      P>

## Univariate Analysis

### Plate failure

In [222]:
chi2_test('plate_failure', df_df)

Absolute frequencies:
               Mix  Reco  Overall
plate_failure                    
False           13    24       37
True             1     0        1

Relative probabilities:
                Mix   Reco  Overall
plate_failure                      
False          92.9  100.0     97.4
True            7.1    0.0      2.6
None
p-value: 0.19043026382552042


### Any complication

In [223]:
chi2_test('any_complication', df_df)

Absolute frequencies:
                  Mix  Reco  Overall
any_complication                    
True               12    18       30
False               2     6        8

Relative probabilities:
                   Mix  Reco  Overall
any_complication                     
True              85.7  75.0     78.9
False             14.3  25.0     21.1
None
p-value: 0.44062934980147084


### Soft tissue complication

In [224]:
chi2_test('soft_tissue_complication', df_df)

Absolute frequencies:
                          Mix  Reco  Overall
soft_tissue_complication                    
True                        8    15       23
False                       6     9       15

Relative probabilities:
                           Mix  Reco  Overall
soft_tissue_complication                     
True                      57.1  62.5     60.5
False                     42.9  37.5     39.5
None
p-value: 0.7477680079698745


### Nonunion

In [225]:
chi2_test('nonunion', df_df)

Absolute frequencies:
          Mix  Reco  Overall
nonunion                    
True        9    10       19
False       5     9       14

Relative probabilities:
           Mix  Reco  Overall
nonunion                     
True      64.3  52.6     57.6
False     35.7  47.4     42.4
None
p-value: 0.5097304608361317


### Wound infection

In [226]:
chi2_test('wound_infection', df_df)

Absolute frequencies:
                 Mix  Reco  Overall
wound_infection                    
False              7    18       25
True               7     6       13

Relative probabilities:
                  Mix  Reco  Overall
wound_infection                     
False            50.0  75.0     65.8
True             50.0  25.0     34.2
None
p-value: 0.1220535573278638


### Plate exposure

In [227]:
chi2_test('complication_plate___exposure', df_df)

Absolute frequencies:
                               Mix  Reco  Overall
complication_plate___exposure                    
False                            8    18       26
True                             6     6       12

Relative probabilities:
                                Mix  Reco  Overall
complication_plate___exposure                     
False                          57.1  75.0     68.4
True                           42.9  25.0     31.6
None
p-value: 0.2596533263226399
